In [ ]:
import argparse
import multiprocessing
import os

import numpy as np
import yaml

from config import SPECS_COMPLETE_YAMLS, MV_RESULTS_DIR
from model import DC, LPAC, Solution
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f = 0
    r_hat = 3
    casestudy = 'imelda'
    pftype = 'dc'
    approach = 'robust'
    timelimit = 10 * 60
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--f')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    parser.add_argument('--timelimit')
    args = parser.parse_args()
    f = int(args.f)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    timelimit = int(args.timelimit)

In [ ]:
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype, 'mv']) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = approach
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# get the appropriate model class
modelcls = model_classes[pftype]

# instantiate a model
MODEL = modelcls(**specs)

# budget constraint
MODEL.con_resource_hi.RHS = f

# update
MODEL.update()

In [ ]:
# if solving an LPAC variant and DC solution is available, load it
zipfile = os.path.join(MV_RESULTS_DIR, f'{casestudy}-dc-f{f}-r{r_hat}.zip')
if os.path.exists(zipfile):
    dc_solution = Solution.from_zip(zipfile)
    x = dc_solution['x'].loc[dc_solution['x'] == 1].index
    for (k, r) in x:
        MODEL.x[k, r].Start = 1

In [ ]:
MODEL.model.setParam('MIPGap', 0.00)
MODEL.model.setParam('TimeLimit', timelimit)
MODEL.model.setParam('Threads', np.floor(multiprocessing.cpu_count()))
MODEL.model.setParam('ImpliedCuts', 2)
MODEL.model.setParam('PreSolve', 2)
MODEL.model.setParam('CutPasses', 10)
MODEL.model.setParam('MIPFocus', 2)
MODEL.update()
MODEL.solve()

os.makedirs(MV_RESULTS_DIR, exist_ok=True)
zipfile = os.path.join(MV_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
sol = Solution.from_solved_instance(MODEL)
sol.to_zip(zipfile, variables=['x', 'ObjBound', 'ObjVal', 'gamma', 'gamma_under', 'gamma_over'])